In [ ]:
import pandas as pd
from simulate_battery import simulate_battery_dispatch
from financial_model import compute_financials


def evaluate_solution(
    params: dict,
    pv_profile: pd.Series,
    demand_profile: pd.Series
) -> list:
    """
    Evaluate a candidate for NSGA-II:

    Parameters
    ----------
    params : dict
        Decision variables:
          - 'pv_kw'       : float, PV capacity (kW) used to compute CAPEX
          - 'battery_kwh' : float, battery capacity (kWh)
    pv_profile : pd.Series
        Half-hourly PV generation (kWh) for 30-year horizon.
    demand_profile : pd.Series
        Half-hourly demand (kWh) for 30-year horizon.

    Returns
    -------
    list
        [ -IRR, NPC ] (NPC = net present cost) for minimisation by NSGA-II.
    """
    # Extract generation series if passed as DataFrame
    if hasattr(pv_profile, 'columns'):
        gen_series = pv_profile['simulated_kwh']
    else:
        gen_series = pv_profile

    # 1) Dispatch battery with PV & demand
    dispatch_df, totals = simulate_battery_dispatch(
        pv_gen=gen_series,
        demand=demand_profile,
        battery_kwh=params['battery_kwh'],
        battery_kw=params.get('battery_kw', None)
    )

    # 2) Compute financials (including IRR + NPC)
    fin = compute_financials(
        totals,
        battery_kwh=params['battery_kwh'],
        pv_kw=params['pv_kw']
    )

    # 3) Extract objectives: maximize IRR, minimize NPC
    irr = fin['irr'] or 0.0
    npc = fin['net_cost']
    return [-irr, npc]
